In [12]:
import pandas as pd
import numpy as np
from load_data import load_financial_data, load_sentiment_data
from preprocessing import calculate_financial_change, aggregate_sentiment, aggregate_financials, merge_sentiment_financials
from sklearn.metrics import mean_squared_error

In [2]:
#tickers = ['ADA', 'BCH', 'BCN', 'BNB', 'BTC', 'DASH', 'EOS', 'ETC', 'ETH', 'ICX', 'IOT', 'LTC', 'NEO', 'QTUM',
#               'TRON', 'VEN', 'XEM', 'XLM', 'XMR', 'XRP', 'ZEC']

tickers = 'BTC'

In [3]:
df_financials = load_financial_data(filepath = 'data/top20_aggre_price_reduced.csv', start_date = '2017-07-28', end_date = '2018-07-27', tickers = tickers)
df_sentiment = load_sentiment_data(filepath = 'data/tweet_amounts_and_sentiment_BTC.csv', start_date = '2017-07-28', end_date = '2018-07-27')


In [4]:
#aggr_df_sentiment = aggregate_sentiment(df_sentiment)
#aggr_df_financials = aggregate_financials(df_financials)
#df_merged = merge_sentiment_financials(aggr_df_sentiment, aggr_df_financials)
df_merged = merge_sentiment_financials(df_sentiment, df_financials)
df_merged = df_merged.drop('BTC', axis=1)

In [5]:
#checking stationarity HOW TO INTERPRET
from statsmodels.tsa.vector_ar.vecm import coint_johansen
coint_johansen(df_merged,-1,1).eig

array([  7.71752268e-02,   3.72368291e-02,   2.07969681e-05])

In [6]:
#creating the train and validation set
train = df_merged[:int(0.9*(len(df_merged)))]
valid = df_merged[int(0.9*(len(df_merged))):]

#fit the model
from statsmodels.tsa.vector_ar.var_model import VAR

model = VAR(endog=train)
model_fit = model.fit()

# make prediction on validation
prediction = model_fit.forecast(model_fit.y, steps=len(valid))

In [13]:
#converting predictions to dataframe
cols = df_merged.columns
#converting predictions to dataframe
pred = pd.DataFrame(index=range(0,len(prediction)), columns=[cols])
for j in range(0,3):
    for i in range(0, len(prediction)):
       pred.iloc[i][j] = prediction[i][j]

#check rmse
for i in cols:
    print('rmse value for', i, 'is : ', np.sqrt(mean_squared_error(pred[i], valid[i])))

rmse value for amount_of_tweets is :  58.2974622814
rmse value for average_sentiment is :  0.122456200835
rmse value for financial_value is :  666.972437441


In [14]:
# Comparing prediction to expected value
exp_value = df_merged['financial_value'].tail(len(valid)).reset_index().drop('index', axis = 1)
exp_value.columns = ['expected_value']
pred = pred.drop(['amount_of_tweets', 'average_sentiment'], axis = 1)
pred.columns = ['predicted_value']
output = pd.concat([pred, exp_value], axis=1)

In [15]:
# calculate error
error_list=[]
for i in range(len(output.index)):
    error = (abs(output.predicted_value[i] - output.expected_value[i])) / output.expected_value[i] * 100
    output.at[i, 'error'] = error



In [16]:
#make final predictions
model = VAR(endog=df_merged)
model_fit = model.fit()
yhat = model_fit.forecast(model_fit.y, steps=1)
print(yhat)

[[  1.89682876e+02   5.29390913e-02   8.20399147e+03]]


In [ ]:
output